In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

# load data & 前處理

In [2]:
df = pd.read_csv("train.csv")

df = df.drop(['Name', 'Cabin', 'Ticket', 'PassengerId', 'Embarked'], axis=1)
df = df.dropna()
df['Age'] = (df['Age'] - df['Age'].min())/( df['Age'].max() - df['Age'].min())
df['Fare'] = (df['Fare'] - df['Fare'].min())/( df['Fare'].max() - df['Fare'].min())
df["Sex"] = df["Sex"].map({'male':0, 'female':1})
df['Sex'] = df['Sex'].astype('object')
df['Pclass'] = df['Pclass'].astype('object')
df = pd.get_dummies(df,prefix=["Sex", "Pclass"])
print(df)
train, vali = train_test_split(df, test_size=0.2, random_state=42)

X_train = train.drop(['Survived'], 1)
X_train = X_train.to_numpy()
X_vali = vali.drop(['Survived'], 1)
X_test = X_vali.to_numpy()
y_train = train['Survived']
y_vali = vali['Survived']

     Survived       Age  SibSp  Parch      Fare  Sex_1  Sex_2  Sex_3  \
0           0  0.271174      1      0  0.014151      0      0      1   
1           1  0.472229      1      0  0.139136      1      0      0   
2           1  0.321438      0      0  0.015469      0      0      1   
3           1  0.434531      1      0  0.103644      1      0      0   
4           0  0.434531      0      0  0.015713      0      0      1   
..        ...       ...    ...    ...       ...    ...    ...    ...   
885         0  0.484795      0      5  0.056848      0      0      1   
886         0  0.334004      0      0  0.025374      0      1      0   
887         1  0.233476      0      0  0.058556      1      0      0   
889         1  0.321438      0      0  0.058556      1      0      0   
890         0  0.396833      0      0  0.015127      0      0      1   

     Pclass_0  Pclass_1  
0           1         0  
1           0         1  
2           0         1  
3           0         1  
4    

In [3]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.BatchNormalization(),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(2, activation="softmax")
])

In [4]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer='adam',
              metrics=["accuracy"])

In [5]:
history = model.fit(X_train, y_train, epochs=15, verbose=1,
                    validation_data=(X_vali, y_vali))
model.save('model.h5')

Epoch 1/15
18/18 [==============================] - 11s 362ms/step - loss: 0.6818 - accuracy: 0.6994 - val_loss: 0.6546 - val_accuracy: 0.6084
Epoch 2/15
18/18 [==============================] - 1s 33ms/step - loss: 0.4920 - accuracy: 0.8176 - val_loss: 0.6503 - val_accuracy: 0.6084
Epoch 3/15
18/18 [==============================] - 1s 35ms/step - loss: 0.3899 - accuracy: 0.8369 - val_loss: 0.6456 - val_accuracy: 0.6084
Epoch 4/15
18/18 [==============================] - 1s 29ms/step - loss: 0.3620 - accuracy: 0.8550 - val_loss: 0.6510 - val_accuracy: 0.6084
Epoch 5/15
18/18 [==============================] - 1s 38ms/step - loss: 0.4230 - accuracy: 0.8277 - val_loss: 0.6377 - val_accuracy: 0.6294
Epoch 6/15
18/18 [==============================] - 1s 35ms/step - loss: 0.3652 - accuracy: 0.8644 - val_loss: 0.6439 - val_accuracy: 0.6084
Epoch 7/15
18/18 [==============================] - 1s 32ms/step - loss: 0.3566 - accuracy: 0.8471 - val_loss: 0.6034 - val_accuracy: 0.6853
Epoch 8/15


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 9)                 36        
_________________________________________________________________
dense (Dense)                (None, 32)                320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256       
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
batch_normalization_3 (Batch (None, 128)               5

# Prediction

In [7]:
import pandas as pd
import keras

In [8]:
df = pd.read_csv("test.csv")

df['Age'] = df['Age'].fillna(18)
df['Age'] = (df['Age'] - df['Age'].min())/( df['Age'].max() - df['Age'].min())
df['Fare'] = (df['Fare'] - df['Fare'].min())/( df['Fare'].max() - df['Fare'].min())
X = df.drop(['Name', 'Cabin', 'Ticket', 'PassengerId', 'Embarked'], axis=1)
X["Sex"] = X["Sex"].map({'male':0, 'female':1})
X['Sex'] = X['Sex'].astype('object')
X['Pclass'] = X['Pclass'].astype('object')
X = pd.get_dummies(X,prefix=["Sex", "Pclass"])

X = X.to_numpy()

In [9]:
model = keras.models.load_model('model.h5')

In [10]:
y_pred = np.argmax(model.predict(X), axis=-1)
output = pd.DataFrame({'PassengerId':df.PassengerId,
                       'Survived':y_pred
                        })

In [11]:
output.to_csv('predict.csv')